In [277]:
## load the libraries required for data processing
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [278]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
lemmatizer = WordNetLemmatizer()
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [279]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer

In [280]:
# importing stopwords from nltk module
stop_words = set(stopwords.words('english'))

In [281]:
# loading dataset
data = pd.read_csv(r"d:/datasets/NextLabs datasets/chrome_reviews.csv")

In [282]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7204 entries, 0 to 7203
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               7204 non-null   int64 
 1   Review URL       7204 non-null   object
 2   Text             7203 non-null   object
 3   Star             7204 non-null   int64 
 4   Thumbs Up        7204 non-null   int64 
 5   User Name        7204 non-null   object
 6   Developer Reply  95 non-null     object
 7   Version          7119 non-null   object
 8   Review Date      7204 non-null   object
 9   App ID           7204 non-null   object
dtypes: int64(3), object(7)
memory usage: 562.9+ KB


In [283]:
# here we need data without any Punctuation char and stopwords to get more accurate prediction on sentences
clean_text =[]

textwostopwords=data['Text']

for review in data['Text']:
    review= re.sub(r'[^\w\s]', '', str(review))
    # \s-->includes [ \t\n\r\f\v]
    # \w-->includes only [a-zA-Z0-9_]
    # removes "[","]","^"
    
    review = re.sub(r'\d','',review)
    #include int type [0-9]
    
    review_token = word_tokenize(review.lower().strip()) 
    #convert into lower case and strip leading and tailing spaces followed by spliting sentnece into words

    review_without_stopwords=[]
    
    # Condition to check, If it is not stopword we lemmatize and append into a list
    for token in review_token:
        if token not in stop_words:
            token= lemmatizer.lemmatize(token)
            review_without_stopwords.append(token)
    
    # and added to cleaned_review list 
    cleaned_review = " ".join(review_without_stopwords)
    clean_text.append(cleaned_review)


In [284]:
stop = stopwords.words('english')

In [285]:
# here we filter out tokens without stop words
text = clean_text
text_tokens = word_tokenize(str(text))

tokens_without_sw = [word for word in text_tokens if not word in stop]

 

In [286]:
tokens_without_sw=str(tokens_without_sw)

In [287]:
# after preprocessing on df we use sentiment analysis 
sentence = tokens_without_sw
tokenized_sentence = nltk.word_tokenize(sentence)
sid = SentimentIntensityAnalyzer()

# here we create 3 list according to the sentiment of sentence
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

# we filter sentiments using their compound score and append into respective lists
for word in tokenized_sentence:
    if (sid.polarity_scores(word)['compound']) >= 0.1:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.1:
        neg_word_list.append(word)
    else:        
        neu_word_list.append(word)  

In [288]:
Text = clean_text
# creating Dataframe and appending clean text to df
df = pd.DataFrame()
df['Text']=Text


In [289]:
# for our information we check and print sentiment and compound scores of each sentence
sid = SentimentIntensityAnalyzer()
for sentence in Text:
     print(sentence)
        
     ss = sid.polarity_scores(sentence)
     for k in ss:
         print('{0}: {1}, ' .format(k, ss[k]), end='')
     print()

helpfull aap
neg: 0.0, neu: 1.0, pos: 0.0, compound: 0.0, 
good
neg: 0.0, neu: 0.0, pos: 1.0, compound: 0.4404, 
able update neither able uninstall
neg: 0.0, neu: 1.0, pos: 0.0, compound: 0.0, 
nice app
neg: 0.0, neu: 0.263, pos: 0.737, compound: 0.4215, 
many unwanted ad
neg: 0.487, neu: 0.513, pos: 0.0, compound: -0.2263, 
app good
neg: 0.0, neu: 0.256, pos: 0.744, compound: 0.4404, 
yes yes
neg: 0.0, neu: 0.0, pos: 1.0, compound: 0.6597, 
awesome
neg: 0.0, neu: 0.0, pos: 1.0, compound: 0.6249, 
bad app
neg: 0.778, neu: 0.222, pos: 0.0, compound: -0.5423, 
many time tried update updating whenever try use app freeze go back home
neg: 0.0, neu: 0.909, pos: 0.091, compound: 0.0516, 
good
neg: 0.0, neu: 0.0, pos: 1.0, compound: 0.4404, 
good app
neg: 0.0, neu: 0.256, pos: 0.744, compound: 0.4404, 
app getting update getting open saying u r using old version plz update updating
neg: 0.0, neu: 0.909, pos: 0.091, compound: 0.0772, 
mostly useful
neg: 0.0, neu: 0.256, pos: 0.744, compound: 0

In [290]:
# concatinating all data to one single df
analyzer = SentimentIntensityAnalyzer()
df['rating'] = df['Text'].apply(analyzer.polarity_scores)
df=pd.concat([df.drop(['rating'], axis=1), df['rating'].apply(pd.Series)], axis=1)
### Creating a dataframe.

In [291]:
sentences=df

In [292]:

i = 0

predicted_value = [ ] #empty series to hold our predicted values

while(i<len(sentences)):
    if ((sentences.iloc[i]['compound'] >= 0.4)):
        predicted_value.append('positive')
        i = i+1
    elif ((sentences.iloc[i]['compound'] >= 0) & (sentences.iloc[i]['compound'] < 0.5)):
        predicted_value.append('neutral')
        i = i+1
    elif ((sentences.iloc[i]['compound'] < 0)):
        predicted_value.append('negative')
        i = i+1
## The threshold value will categorize if a given sentence is positive negative or neutral in nature.    

In [293]:
df['Target'] = predicted_value 
## A new column has been created called as 'Target' with sentiments assigned to a given text.

In [294]:
# created df to predict the correctness of prediction method at the end
Prediction_df=df

In [295]:
df.drop(['neg','neu','pos','compound'],axis=1,inplace=True)
## Dropping the neg, neu, pos, and compound columns.

In [296]:
df['Star']=data['Star']

In [297]:
count=df['Text'].count()


In [298]:
# appending reviews where sentiment rating does not match with stars
low_star_reviews = df[df.Star <3 ]
avg_star_reviews = df[df.Star ==3]
high_star_reviews =df[df.Star >3]



In [299]:
low_star_reviews_with_pos_comments = low_star_reviews[low_star_reviews.Target != 'negative']
df1=low_star_reviews_with_pos_comments

In [300]:
avg_star_reviews_with_posorneg_comments = avg_star_reviews[avg_star_reviews.Target != 'neutral']
df2=avg_star_reviews_with_posorneg_comments

In [301]:
high_star_reviews_with_neg_comments=high_star_reviews[high_star_reviews.Target != 'positive']
df3=high_star_reviews_with_neg_comments

In [302]:
resultdf=[high_star_reviews_with_neg_comments,avg_star_reviews_with_posorneg_comments,low_star_reviews_with_pos_comments]

In [303]:
# here we created three diff dataframe where sentiment rating does not match with stars

In [304]:
# and we merging it into single df
dataframe1 = pd.DataFrame(df1)
dataframe2 = pd.DataFrame(df2)
dataframe3 = pd.DataFrame(df3)
merged_df = pd.concat([dataframe1, dataframe2,dataframe3])

In [317]:

text_df=pd.read_csv("d:/datasets/NextLabs datasets/chrome_reviews.csv")
merged_df["Text"]=text_df["Text"]
merged_df

Text    Target  Star
2        Not able to update. Neither able to uninstall.   neutral     1
9     Many times I tried to update its not updating....   neutral     1
12    App is not getting update and it is not gettin...   neutral     1
15                Very coming in real status. Thank you   neutral     1
19    I want to download greana free fire but that n...   neutral     1
...                                                 ...       ...   ...
7187                                             Ponraj   neutral     5
7189                                      Frustration!!  negative     4
7192                                           Excllent   neutral     5
7194                                     shubham gurjar   neutral     5
7199                                          Bagusss..   neutral     5

[3496 rows x 3 columns]

In [306]:
#now for predictions we use tfidf vectorization
# which Convert a collection of raw documents to a matrix of TF-IDF features
tfidf =TfidfVectorizer()
X = tfidf.fit_transform(Prediction_df['Text'].values.astype('U'))
Y = Prediction_df['Target'] 

In [307]:
# train test split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=40)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(5763, 4955) (1441, 4955) (5763,) (1441,)


In [308]:
# prediction using LogisticRegression
log_reg = LogisticRegression().fit(X_train,Y_train)

#predict on Train
train_preds = log_reg.predict(X_train)

#Accuracy on Train
print("Model Accuracy on Train is",accuracy_score(Y_train,train_preds))

#predict on Test
test_preds=log_reg.predict(X_test)

#Accuracy on Test
print("Model Accuracy on Test is",accuracy_score(Y_test,test_preds))

Model Accuracy on Train is 0.9316328301231998
Model Accuracy on Test is 0.890353920888272


In [309]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, auc, confusion_matrix, roc_auc_score, roc_curve, recall_score

In [310]:
DT = DecisionTreeClassifier().fit(X,Y)

In [311]:
#predict on train 
train_preds2 = DT.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds2))

#predict on test
test_preds2 = DT.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds2))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds2))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds2))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds2).sum(),'/',((Y_test == test_preds2).sum()+(Y_test != test_preds2).sum()))
print('-'*50)

# Kappa Score
print('KappaScore is: ', metrics.cohen_kappa_score(Y_test,test_preds2))

Model accuracy on train is:  0.9994794377928162
Model accuracy on test is:  0.9986120749479528
--------------------------------------------------
confusion_matrix train is:  [[ 641    0    0]
 [   0 2414    0]
 [   0    3 2705]]
confusion_matrix test is:  [[161   0   0]
 [  0 608   0]
 [  0   2 670]]
Wrong predictions out of total
--------------------------------------------------
2 / 1441
--------------------------------------------------
KappaScore is:  0.9976558433107756


In [312]:
#fit the model on train data 
RF=RandomForestClassifier().fit(X_train,Y_train)
#predict on train 
train_preds3 = RF.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds3))

#predict on test
test_preds3 = RF.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds3))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds3))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds3))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds3).sum(),'/',((Y_test == test_preds3).sum()+(Y_test != test_preds3).sum()))
print('-'*50)

Model accuracy on train is:  0.9994794377928162
Model accuracy on test is:  0.8979875086745316
--------------------------------------------------
confusion_matrix train is:  [[ 641    0    0]
 [   0 2414    0]
 [   0    3 2705]]
confusion_matrix test is:  [[ 88  64   9]
 [  6 589  13]
 [  7  48 617]]
Wrong predictions out of total
--------------------------------------------------
147 / 1441
--------------------------------------------------


In [313]:
#fit the model on train data 
KNN = KNeighborsClassifier().fit(X_train,Y_train)
#predict on train 
train_preds4 = KNN.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds4))

#predict on test
test_preds4 = KNN.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds4))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds4))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds4))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds4).sum(),'/',((Y_test == test_preds4).sum()+(Y_test != test_preds4).sum()))
print('-'*50)

Model accuracy on train is:  0.8115564809994794
Model accuracy on test is:  0.8056904927133934
--------------------------------------------------
confusion_matrix train is:  [[ 174  463    4]
 [   5 2393   16]
 [   5  593 2110]]
confusion_matrix test is:  [[ 40 119   2]
 [  2 597   9]
 [  1 147 524]]
Wrong predictions out of total
--------------------------------------------------
280 / 1441
--------------------------------------------------


In [314]:
#fit the model on train data 
SVM = SVC(kernel='linear')
SVM.fit(X_train, Y_train)

#predict on train 
train_preds5 = SVM.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds5))

#predict on test
test_preds5 = SVM.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds5))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds5))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds5))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds5).sum(),'/',((Y_test == test_preds5).sum()+(Y_test != test_preds5).sum()))
print('-'*50)

Model accuracy on train is:  0.9614783966684018
Model accuracy on test is:  0.9090909090909091
--------------------------------------------------
confusion_matrix train is:  [[ 515  114   12]
 [  22 2382   10]
 [  10   54 2644]]
confusion_matrix test is:  [[105  48   8]
 [ 12 589   7]
 [  8  48 616]]
Wrong predictions out of total
--------------------------------------------------
131 / 1441
--------------------------------------------------


### After checking all prediction we can see DecisionTreeClassifier() is most accurate than others so we used that in our deployment